# LCOE calculations for different standalone power generation technologies
This code calculates the Levelised Cost of Energy (LCOE) of different standalone power generation tecnologies, based on Geografic Information Systems (GIS) data. The technologies considered are:
* Wind power
* Photovoltaic solar power
* Diesel generators

In [127]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from math import pi, exp, log, sqrt
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from openpyxl import load_workbook
import os

In [128]:
#Define the variables for the analysis

scenario = 'phase2_scenario1_part3'

CAPEX_level = [1,3]
wind_CAPEX_level = CAPEX_level
pv_CAPEX_level = CAPEX_level
fuel_cost_level = [1,3]
environmental_cost_level = [1,3]
discount_rate = 0.08

#Diesel input values
diesel_emission_fac = 2.69719 #kgCO2/l
diesel_fuel_req = 0.095 #fuel requirementes l/kWh
diesel_cf = 0.5 #capacity factor

#Gasoil input values
Gasoil_emission_fac = 2.69719 #kg CO2/m3
diesel_fuel_req=0.095 #fuel requirementes m3/kWh
gasoil_cf = 0.5 #capacity factor

#50Diesel 50 Gasoil input values
diesel_gasoil_emission_fac = 2.69719 #kg CO2/l
diesel_gasoil_fuel_req=0.095 #fuel requirementes l/kWh
diesel_gasoil_cf = 0.5 #capacity factor

#Grid emission factor
grid_emission_fac = 0.5124661 #kg CO2/kWh needs to be defined, this is a dummy value

wind_CAPEX = {1:3000,2:2500,3:1500} # $/kW
pv_CAPEX = {1:2800,2:2000,3:1200} # $/kW
diesel_fuel_cost = {1:'Dprice1',2:'Dprice2',3:'Dprice3'}
# gasoil_fuel_cost = {1:'Gasoil1',2:'Gasoil2',3:'Gasoil3'}
diesel_gasoil_fuel_cost = {1:'Diesel&Gasoil1',2:'Diesel&Gasoil2',3:'Diesel&Gasoil3'}
environmental_cost = {1:0,2:0.22,3:0.33} # $/kg/CO2

desalination = 'y'#input(str('Account for desalination energy?(y/n) '))

#Define the levels of the sensitivity variables to use
sensitivity_vars = {'wind_CAPEX':wind_CAPEX, 'pv_CAPEX':pv_CAPEX,'environmental_cost':environmental_cost}

#comparison variables
grid_price_dic = {1:'Epricelow',2:'Epricemid',3:'Epricehigh'} #change the names of the electricity price levels so they are consistent wit the ones in the excel sheet
grid_price = 'Epricelow' #no need to change this, is only to inicialize the variable
diesel_lcoe = 'diesel_lcoe' #this cannot be changed
pv_lcoe = 'pv_lcoe' #this cannot be changed
wind_lcoe = 'wind_lcoe' #this cannot be changed
diesel_gasoil_lcoe = 'diesel_gasoil_lcoe' #this cannot be changed
gasoil_lcoe = 'gasoil_lcoe' #this cannot be changed
def eval_dict(comp_vars):
    new_dict = {}
    for key, item in comp_vars.items():
        new_dict[key] = [eval(x) for x in item]
    return new_dict

# the following dictionary tells the model which parameters should compare in each country, the values inside the lists, can be changed but need to match the variables efined above
comp_vars = {'Algeria':['diesel_lcoe','pv_lcoe','wind_lcoe'],
            'Tunisia':['diesel_gasoil_lcoe','pv_lcoe','wind_lcoe'],
            'Libya':['grid_price','pv_lcoe','wind_lcoe']}

#df1=pd.read_excel('phase2_waterdemand_20190607.xlsx') #This is the output file from code2. 
df1=pd.read_csv('phase2_scenario1_part2.csv')
if desalination == 'y':
    scenario += 'Des'

In [129]:
grid_price_dic = {1:'Epricelow',2:'Epricemid',3:'Epricehigh'}
grid_price = 'Epricelow'
diesel_lcoe = 'diesel_lcoe'
pv_lcoe = 'pv_lcoe'
wind_lcoe = 'wind_lcoe'
diesel_gasoil_lcoe = 'diesel_gasoil_lcoe'
def eval_dict(comp_vars):
    new_dict = {}
    for key, item in comp_vars.items():
        new_dict[key] = [eval(x) for x in item]
    return new_dict
comp_vars = {'Algeria':['diesel_lcoe','pv_lcoe','wind_lcoe'],
            'Tunisia':['diesel_gasoil_lcoe','pv_lcoe','wind_lcoe'],
            'Libya':['grid_price','pv_lcoe','wind_lcoe']}

In [130]:
#Introduce a new dataframe which includes the electricity and diesel cost for each county
# Low:1, Medium:2 and High:3

df2=pd.read_excel('DEprices3_20190603.xlsx')
df2.round(decimals=3)

,country,Dprice1,Dprice2,Dprice3,Epricelow,Epricemid,Epricehigh,Gasoline1,Gasoline2,Gasoline3,Gasoil1,Gasoil2,Gasoil3,Diesel&Gasoil1,Diesel&Gasoil2,Diesel&Gasoil3
0,Algeria,0.17,0.213,0.255,0.028,0.035,0.042,0.28,0.350,0.420,0.192,0.240,0.288,0.192,0.240,0.288
1,Tunisia,0.62,0.775,0.930,0.100,0.125,0.150,0.73,0.912,1.095,0.500,0.625,0.750,0.500,0.625,0.750
2,Libya,0.11,0.138,0.165,0.168,0.210,0.252,0.11,0.138,0.165,NaN,0.000,0.000,1.000,1.250,1.500


In [131]:
#Merging both dataframes on the country name since each country has different cost for electricity and diesel: 
data = pd.merge(df1, df2, on='country')

In [132]:
#Setting the default value for these parameters

for i in range (1,13):
    data['PD_E_{}'.format(i)]=0      #PD_E_: Peak Demand (kw)
    data['ED_E_{}'.format(i)]=0      #ED_E_: Electricity Demand (kwh)

With the following expresion the total dynamic head ($tdh$) can be calculated: 

$$tdh = gw\_depth + wdd + oap + pld$$  
Where:  
* $tdh$: total dynamic head (m)  
* $gw\_depth$: depth to groundwater (m)  
* $wdd$: water drawdown (m)  
* $oap$: operating application pressure (m)  
* $pld$: pressure loss in distribution (m)  

Operating application pressure: 0 m (SU), 30 m (SP), 10 m (DR) , check if it is in 1992?    
Pressure loss in distributionL: 1 m (SU), 20% of oap (SP, DR) , check if it is in 1992?

In [133]:
wdd=0   #water drawdown
oap=0   #operating application pressure (m) 
pld=0   #pressure loss in distribution

#This step of ground water level specification was removed since we have a layer specifying the depth in m:

#data.rename(index=str, columns={'gw_depth': 'gw_m'})

def tdh_gw(row):
    tdh=(row['gwdepth']+wdd+oap+pld)
    return tdh


data['tdh_gw'] = data.apply(tdh_gw , axis=1)

### Pumping plant efficiency:
$$\text{pumping plant efficiency (%)}= \text{fuel efficiency (%)} \times \text{power unit eff (%)} \times  \text{transmission eff (%)} \times \text{pump eff (%)} \times 100\%$$

**The Power Unit:** can be diesel engine or electric engine. In the first we call it (diesel powered pump) and the second (electric powered pump).
 
### Value obtained from FAO1992, pages 26-27:

#### Diesel powered pump:
* **Worst case:** $0.9\times 0.3\times 0.9\times 0.4 = 10\%\ (0.1)$
* **Best case:** $1\times 0.4\times 1\times 0.8 = 32\%\ (0.32)$

#### Electric powered pump:
* **Worst case:** $0.9\times 0.75\times 0.9\times 0.4 = 25\%\ (0.25)$
* **Best case:** $1\times 0.85\times 1\times0.8 = 70\%\ (0.7)$

In [134]:
Epump_plant_eff=0.36 #transmission eff * pump eff - the efficiecy of the fuel and the power generation unit are accounted in the LCOE method

for i in range (1,13):
    PD_E = 'PD_E_{}'.format(i)
    PWD = 'PWD_{}'.format(i)
    SSWD = 'SSWD_{}'.format(i)
    ED_E = 'ED_E_{}'.format(i)
    
    data[PD_E]=(9.81*(data[PWD]/1000)*data['tdh_gw'])/Epump_plant_eff
    data[ED_E]=(data[SSWD]*data['tdh_gw']*0.00272)/Epump_plant_eff
    
    if desalination == 'y':
        data[PD_E] += (data[PWD]*data['Einten_KWh/m3']*3600/1000)
        data[ED_E] += (data['Edesal_GWh_{}'.format(i)]*1000000)

## Calculate the wind capacity factor based on the average wind velocity:

In [135]:
for i in range (1,13):
    data['wind_cf_{}'.format(i)]=0 

mu = 0.97  # availability factor
t = 24*30
p_rated = 600
z = 55  # hub height
zr = 80  # velocity measurement height
es = 0.85  # losses in wind electricity
u_arr = range(1, 26)
p_curve = [0, 0, 0, 0, 30, 77, 135, 208, 287, 371, 450, 514, 558,
           582, 594, 598, 600, 600, 600, 600, 600, 600, 600, 600, 600]

def get_wind_cf(df, wind):
    u_zr = df[wind]

    # Adjust for the correct hub height
    alpha = (0.37 - 0.088 * np.log(u_zr)) / (1 - 0.088 * np.log(zr / 10))
    u_z = u_zr * (z / zr) ** alpha

    # Rayleigh distribution and sum of series
    rayleigh = [(pi / 2) * (u / u_z ** 2) * np.exp((-pi / 4) * (u / u_z) ** 2) for u in u_arr]
    energy_produced = np.array(sum([mu * es * t * p * r for p, r in zip(p_curve, rayleigh)]))

    return energy_produced/(p_rated * t)

for i in range (1,13):
    wind='wind{}'.format(i)
    data['wind_cf_{}'.format(i)] = get_wind_cf(data, wind)
    data['pv_cf_{}'.format(i)] = data['srad{}'.format(i)] / (60*60*24) # solar rad: (kJ/(m2.day))*30.day/month*1h/(60*60s) = kWh/(m2.month)*30/(60*60)
                                                                        # pv_cf: energy produced / (p_rated * t) :  kWh/(m2.month)*30/(60*60) / (1kW*(24*30)h/m2.month) = srad / (60*60*24)

In [136]:
def get_installed_capacity(df,capacity_factor,peak_demand):
    return df[peak_demand] / df[capacity_factor]

for i in range (1,13):
    data['wind_installed_capacity_{}'.format(i)] = get_installed_capacity(data,'wind_cf_{}'.format(i),'PD_E_{}'.format(i))
    data['pv_installed_capacity_{}'.format(i)] = get_installed_capacity(data,'pv_cf_{}'.format(i),'PD_E_{}'.format(i))
    data['diesel_installed_capacity_{}'.format(i)] = data['PD_E_{}'.format(i)] / diesel_cf
#     data['gasoil_installed_capacity_{}'.format(i)] = data['PD_E_{}'.format(i)] / gasoil_cf
    data['diesel_gasoil_installed_capacity_{}'.format(i)] = data['PD_E_{}'.format(i)] / diesel_gasoil_cf
    
data['max_capacity_wind'] = data.filter(like='wind_installed_capacity_').max(axis=1)
data['max_capacity_pv'] = data.filter(like='pv_installed_capacity_').max(axis=1)
data['max_capacity_diesel'] = data.filter(like='diesel_installed_capacity_').max(axis=1)
# data['max_capacity_gasoil'] = data.filter(like='gasoil_installed_capacity_').max(axis=1)
data['max_capacity_diesel_gasoil'] = data.filter(like='diesel_gasoil_installed_capacity_').max(axis=1)

In [137]:
data['total_demand'] = data.filter(like='ED_E_').sum(axis=1)

In [138]:
def get_fuel_cost(fuel_cost,el_gen,efficiency,fuel_req):
    if type(fuel_cost) == int:
        fuel = el_gen * fuel_req * fuel_cost / efficiency
        fuel[0] = 0
    else:
        fuel = np.array([gen * fuel_req * cost / efficiency for gen,cost in zip(el_gen,fuel_cost)])
    return fuel

In [139]:
def get_emissions(el_gen,efficiency,fuel_req,emission_factor):
    emissions = el_gen * fuel_req * emission_factor / efficiency
    return emissions

In [140]:
def get_lcoe(max_capacity,total_demand,tech_life,om_cost,capital_cost,discount_rate,
             fuel_cost=0,fuel_req=0,efficiency=1,emission_factor=0,env_cost=0):
    # Perform the time-value LCOE calculation
    project_life = end_year - start_year
    reinvest_year = 0
    capital_cost = capital_cost * max_capacity
    om_cost = om_cost * capital_cost

    # If the technology life is less than the project life, we will have to invest twice to buy it again
    if tech_life < project_life:
        reinvest_year = tech_life

    year = np.arange(project_life)
    el_gen = np.array([demand * np.ones(project_life -1) for demand in total_demand])
    el_gen = np.insert(el_gen,0,0,axis=1)
    discount_factor = np.array([(1 + discount_rate) ** year for i in total_demand])
    investments = np.array([np.zeros(project_life-1) for i in capital_cost])
    investments = np.insert(investments,0,capital_cost,axis=1)
    
    if reinvest_year:
        investments = np.delete(investments,reinvest_year,axis=1)
        investments = np.insert(investments,reinvest_year,capital_cost,axis=1)

    salvage = np.array([np.zeros(project_life-1) for i in capital_cost])
    used_life = project_life
    if reinvest_year:
        # salvage will come from the remaining life after the re-investment
        used_life = project_life - tech_life
    salvage = np.insert(salvage,-1,capital_cost * (1 - used_life / tech_life),axis=1)

    operation_and_maintenance = np.array([i * np.ones(project_life-1) for i in om_cost])
    operation_and_maintenance = np.insert(operation_and_maintenance,0,0,axis=1)
    
    fuel = get_fuel_cost(fuel_cost,el_gen,efficiency,fuel_req)
    emissions = get_emissions(el_gen,efficiency,fuel_req,emission_factor)
    
    discounted_costs = (investments + operation_and_maintenance + fuel + emissions*env_cost - salvage) / discount_factor
    discounted_generation = el_gen / discount_factor
    
    return discounted_costs.sum(axis=1) / discounted_generation.sum(axis=1)

## LCOE calculations
Change factors of the technologies in the next cell, where:
* ``start_year``: starting year of the analysis
* ``end_year``: starting year of the analysis
* ``tech_life``: is the life time of the technology, (years)
* ``om_cost``: operation and maintenance cost, excluding fuel cost, ($\frac{\$}{kWh}$)
* ``capital_cost``: capital cost including installation costs, ($\frac{\$}{kW}$)
* ``discount_rate``: discount rate of the project, (\%)
* ``efficiency``: efficiency of the fuel and power unit, (\%)
* ``fuel_cost``: cost of the fuel used, ($\frac{\$}{l}$)
* ``fuel_req``: liters of fuel needed to produce one kWh of energy, ($\frac{l}{kWh}$)
* ``emission_factor``: CO2 emission factor of the fuel, ($\frac{kgCO_2}{l}$)
* ``env_cost``: environmental cost per emitted kg of CO2, ($\frac{$}{kgCO_2}$)

In [141]:
def get_least_cost(country, variables, j, k):
    data[grid_price_dic[j]] += environmental_cost[k] * grid_emission_fac
    data.loc[data['country']==country,'least_cost_technology'] = data.loc[data['country']==country,variables].idxmin(axis=1)
    wind = pd.DataFrame()
    wind['Demand'] = data.groupby('NAME_1')['total_demand'].sum()
    wind[['max_capacity_wind','total_demand']] = data.loc[data['least_cost_technology'] == 'wind_lcoe',['NAME_1','max_capacity_wind','total_demand']].groupby('NAME_1').sum()
    wind.fillna(0,inplace=True)
    pv = pd.DataFrame()
    pv['Demand'] = data.groupby('NAME_1')['total_demand'].sum()
    pv[['max_capacity_pv','total_demand']] = data.loc[data['least_cost_technology'] == 'pv_lcoe',['NAME_1','max_capacity_pv','total_demand']].groupby('NAME_1').sum()
    pv.fillna(0,inplace=True)
    diesel = pd.DataFrame()
    diesel['Demand'] = data.groupby('NAME_1')['total_demand'].sum()
    diesel[['max_capacity_diesel','total_demand']] = data.loc[data['least_cost_technology'] == 'diesel_lcoe',['NAME_1','max_capacity_diesel','total_demand']].groupby('NAME_1').sum()
    diesel.fillna(0,inplace=True)
#     gasoil = pd.DataFrame()
#     gasoil['Demand'] = data.groupby('NAME_1')['total_demand'].sum()
#     gasoil[['max_capacity_gasoil','total_demand']] = data.loc[data['least_cost_technology'] == 'gasoil_lcoe',['NAME_1','max_capacity_gasoil','total_demand']].groupby('NAME_1').sum()
#     gasoil.fillna(0,inplace=True)
    diesel_gasoil = pd.DataFrame()
    diesel_gasoil['Demand'] = data.groupby('NAME_1')['total_demand'].sum()
    diesel_gasoil[['max_capacity_diesel_gasoil','total_demand']] = data.loc[data['least_cost_technology'] == 'diesel_gasoil_lcoe',['NAME_1','max_capacity_diesel_gasoil','total_demand']].groupby('NAME_1').sum()
    diesel_gasoil.fillna(0,inplace=True)
    grid = pd.DataFrame()
    grid['Demand'] = data.groupby('NAME_1')['total_demand'].sum()
    grid['total_demand'] = data.loc[data['least_cost_technology'] == grid_price,['NAME_1','total_demand']].groupby('NAME_1').sum()
    grid.fillna(0,inplace=True)
    
    return wind, pv, diesel, diesel_gasoil, grid

In [142]:
def pumping_cost(country, variables):
    data.loc[data['country']==country,'Pumping cost (M$)'] = data.loc[data['country']==country,variables].min(axis=1) * data.loc[data['country']==country,'total_demand']
    

In [143]:
def get_summary():
    final=data.groupby('NAME_1').sum()
    summary = pd.DataFrame({'Irrigated area (ha)':final['area_ha'],
                            'Water intensity (m3/ha)':final.filter(like='SSWD_').sum(axis=1)/final['area_ha'],
                            'Water demand (Mm3)':final.filter(like='SSWD_').sum(axis=1)/1000000,
#                             'Wind installed capacity (MW)':wind['max_capacity_wind'],
#                             'Solar installed capacity (MW)':pv['max_capacity_pv'],
#                             'Diesel installed capacity (MW)':diesel['max_capacity_diesel'],
                            'Total demand (GWh)':final['total_demand']/1000000})

    summary['PV Generation share (%)'] = [i/t*100 for i, t in zip(pv['total_demand'],final['total_demand'])]
    summary['Wind Generation share (%)'] = [i/t*100 for i, t in zip(wind['total_demand'],final['total_demand'])]
    summary['Diesel Generation share (%)'] = [i/t*100 for i, t in zip(diesel['total_demand'],final['total_demand'])]
    summary['Diesel&Gasoil Generation share (%)'] = [i/t*100 for i, t in zip(diesel_gasoil['total_demand'],final['total_demand'])]
    summary['Grid share (%)'] = [i/t*100 for i, t in zip(grid['total_demand'],final['total_demand'])]
    summary['Wind average LCOE ($/kWh)'] = data.groupby('NAME_1')['wind_lcoe'].mean()
    summary['PV average LCOE ($/kWh)'] = data.groupby('NAME_1')['pv_lcoe'].mean()
    summary['Diesel average LCOE ($/kWh)'] = data.groupby('NAME_1')['diesel_lcoe'].mean()
#     summary['Gasoil average LCOE ($/kWh)'] = data.groupby('NAME_1')['gasoil_lcoe'].mean()
    summary['Diesel&Gasoil average LCOE ($/kWh)'] = data.groupby('NAME_1')['diesel_gasoil_lcoe'].mean()
    summary['Country'] = summary.index.map(data.groupby('NAME_1')['country'].first())
    summary['Pumping cost (M$)'] = data.groupby('NAME_1')['Pumping cost (M$)'].sum() / 1000000
                                        #data.loc[data['country']==country].groupby('NAME_1').sum().filter(like='SSWD_').sum(axis=1)
    
    summary['Pumping cost ($/m3)'] = summary['Pumping cost (M$)'] / (summary['Water demand (Mm3)'])
    #display(Markdown('**The following table shows the needed installed capacity\
                    #of each technology type if the total demand were to be covered by such technology,\
                    #and the percentage of adoption of the least cost technology**'))
    #summary.T
    return summary

In [144]:
def save_summary(name): 
    cwd = os.getcwd()
    folder = scenario + ' - results'
    os.chdir(folder)
    try:
        book = load_workbook(scenario+'summary.xlsx')
        writer = pd.ExcelWriter(scenario+'summary.xlsx', engine='openpyxl')
        writer.book = book
    except:
        writer = pd.ExcelWriter(scenario+'summary.xlsx', engine='openpyxl')

    summary.to_excel(writer, sheet_name=name)

    writer.save()
    writer.close()
    os.chdir(cwd)

In [145]:
def save_data(name):
    #Create a Pandas Excel writer using XlsxWriter as the engine.
    cwd = os.getcwd()
    folder = scenario + ' - results' 
    if not os.path.exists(folder):
        os.makedirs(folder)
    os.chdir(folder)
    data.to_csv(name + '.csv',index=False)
    #writer = pd.ExcelWriter(name + '.xlsx', engine='xlsxwriter')
    #writer.book.use_zip64() 

    # Convert the dataframe to an XlsxWriter Excel object.
    #data.to_excel(writer, sheet_name='sur_ref')

    # Close the Pandas Excel writer and output the Excel file.
    #writer.save()
    #writer.close()
    os.chdir(cwd)

In [146]:
start_year = 2016
end_year = 2030

for i in CAPEX_level:
    for j in fuel_cost_level:
        for k in environmental_cost_level:
            data['wind_lcoe'] = get_lcoe(max_capacity = data['max_capacity_wind'],total_demand=data['total_demand'],
                                tech_life=20,om_cost=0.02,capital_cost=sensitivity_vars['wind_CAPEX'][i],discount_rate=discount_rate, efficiency=1)

            data['pv_lcoe'] = get_lcoe(max_capacity = data['max_capacity_pv'],total_demand=data['total_demand'],
                                tech_life=15,om_cost=0.01,capital_cost=sensitivity_vars['pv_CAPEX'][i],discount_rate=discount_rate, efficiency=1)
             
            # for the diesel generators calculation, also provide fuel requirements (l/kwh), the efficiency, 
            # the emission factor of (kgCO2 / liter) and the environmental cost ($/kgCO2)
            data['diesel_lcoe'] = get_lcoe(max_capacity = data['max_capacity_diesel'],total_demand=data['total_demand'],
                                tech_life=10,om_cost=0.1,capital_cost=938,discount_rate=discount_rate,fuel_cost=data[diesel_fuel_cost[j]],
                                fuel_req=diesel_fuel_req, efficiency=0.27,emission_factor=diesel_emission_fac,env_cost=sensitivity_vars['environmental_cost'][k])
            
#             data['gasoil_lcoe'] = get_lcoe(max_capacity = data['max_capacity_gasoil'],total_demand=data['total_demand'],
#                                 tech_life=10,om_cost=0.1,capital_cost=938,discount_rate=discount_rate,fuel_cost=data[gasoil_fuel_cost[j]],
#                                 fuel_req=gasoil_fuel_req, efficiency=0.27,emission_factor=gasoil_emission_fac,env_cost=sensitivity_vars['environmental_cost'][k])
            
            data['diesel_gasoil_lcoe'] = get_lcoe(max_capacity = data['max_capacity_diesel_gasoil'],total_demand=data['total_demand'],
                                tech_life=10,om_cost=0.1,capital_cost=938,discount_rate=discount_rate,fuel_cost=data[diesel_gasoil_fuel_cost[j]],
                                fuel_req=diesel_gasoil_fuel_req, efficiency=0.27,emission_factor=diesel_gasoil_emission_fac,env_cost=sensitivity_vars['environmental_cost'][k])
            #,env_cost=220/1000
            grid_price = grid_price_dic[j]
            for country, variables in eval_dict(comp_vars).items():
                wind, pv, diesel, diesel_gasoil, grid = get_least_cost(country, variables, j, k)
                pumping_cost(country, variables)
            summary = get_summary()
            name = scenario + '_' + 'CAPEX_' + str(i) + '_FEnv_' + str(j) + str(k)
            
            save_data(name)
            save_summary(name)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


In [147]:
summary

,Irrigated area (ha),Water intensity (m3/ha),Water demand (Mm3),Total demand (GWh),PV Generation share (%),Wind Generation share (%),Diesel Generation share (%),Diesel&Gasoil Generation share (%),Grid share (%),Wind average LCOE ($/kWh),PV average LCOE ($/kWh),Diesel average LCOE ($/kWh),Diesel&Gasoil average LCOE ($/kWh),Country,Pumping cost (M$),Pumping cost ($/m3)
NAME_1,,,,,,,,,,,,,,,,
Adrar,24101.816848,19245.036317,463.840341,142.795783,100.000000,0.0,0.0,0.0,0.000000,6.791567,0.734420,1.110819,1.122431,Algeria,104.004450,0.224225
Beni Walid,8557.616421,18868.686141,161.470978,66.168982,33.713601,0.0,0.0,0.0,66.286399,13.169070,1.282514,1.479985,1.949707,Libya,82.277733,0.509551
Biskra,5749.995666,12392.888756,71.259057,51.948473,100.000000,0.0,0.0,0.0,0.000000,14.415216,0.886525,1.191393,1.203004,Algeria,46.221916,0.648646
Djelfa,3449.972096,12659.454676,43.674765,3.861052,100.000000,0.0,0.0,0.0,0.000000,7.973748,0.886869,1.209232,1.220843,Algeria,3.364518,0.077036
El Oued,68999.985344,14154.960584,976.692073,616.380541,100.000000,0.0,0.0,0.0,0.000000,10.984896,0.873593,1.185412,1.197023,Algeria,537.489108,0.550316
Gabes,2874.983928,12025.650107,34.573551,25.754501,100.000000,0.0,0.0,0.0,0.000000,9.998620,0.895886,1.421516,1.358183,Tunisia,23.084849,0.667703
Ghardaia,34499.998422,15679.265990,540.934652,102.793743,100.000000,0.0,0.0,0.0,0.000000,4.783996,0.814257,1.161730,1.173341,Algeria,82.455997,0.152432
Illizi,1770.396215,17183.912109,30.422333,20.402674,100.000000,0.0,0.0,0.0,0.000000,6.410884,0.786953,1.106710,1.118322,Algeria,16.040718,0.527268
Jufrah,7963.713904,15678.350448,124.857897,31.575241,100.000000,0.0,0.0,0.0,0.000000,9.536171,0.785215,1.063342,1.533064,Libya,24.798755,0.198616
